In [1]:
import torch
import mmd
import numpy as np
from mmd import *
#This notebook is meant to be run at the same level as the scripts folder.

In [2]:
#Load some data from storage.
data_all=torch.load("data_all0.pt")
embs_all=torch.load("embs_all0.pt")

In [3]:
#Bandwidth equation copied from:
# https://github.com/GreatA1exander/pt-mmd-demo/blob/master/common/PT-MMD%20Driver%20Demo.ipynb
def determine_bandwidth(data1,data2,metric):
    distances = np.zeros((len(data1),len(data2)))
    for x,point1 in enumerate(data1):
        for y,point2 in enumerate(data2):
            distances[x,y] = metric(point1,point2)
            
    print("Best single KERNEL has bandwidth %.5f" % np.median(distances))
    return np.median(distances)

#If norm fails on execution, it is likely because your x and y are not both 2D.
distance_metric = lambda x,y: np.linalg.norm(x-y,2)


In [4]:
#I have been unable to decipher the meaning of embs, dimensions.
# For data it means there are 5 images, with 10 layers, that are 1984 by 2016 pixels.
#I am not sure where the ten comes from, but the evaluation methods frequently only use the first layer.
print(data_all.shape)
print(embs_all.shape)

torch.Size([5, 10, 1984, 2016])
torch.Size([4, 256, 26, 32])


In [6]:
results = np.zeros(5)
for i in range(5):
    x=data_all[0][0]
    y=x + np.random.normal(0,i/10, x.shape)
    bandwidth = determine_bandwidth(x,y,distance_metric)
    results[i] =mix_rbf_mmd2(x,y, [bandwidth])
results
#Checks that the algorithm is working.

Best single KERNEL has bandwidth 31.35715
Best single KERNEL has bandwidth 31.67813
Best single KERNEL has bandwidth 32.62355
Best single KERNEL has bandwidth 34.13388
Best single KERNEL has bandwidth 36.13698


In [8]:
results = np.zeros(5)
#Experimenting with the embedded image (latent space) using code from
#src.evaluation.methods.cosEmbeddingImage()
for i in range(3):
    embs_now=embs_all[i+1]
    embs_win=embs_all[i]
    embs_now = embs_now[:, :embs_now.shape[1]//2]
    embs_win = embs_win[:, :embs_win.shape[1]//2]
    embs_now = embs_now[0]
    embs_win = embs_win[-1]
    bandwidth = determine_bandwidth(embs_now,embs_win,distance_metric)
    results[i] =mix_rbf_mmd2(embs_now,embs_win, [bandwidth])

Best single KERNEL has bandwidth 20.67920
Best single KERNEL has bandwidth 22.62339
Best single KERNEL has bandwidth 21.88240


In [7]:
results

array([0.00000000e+00, 7.09299153e-05, 9.09960370e-04, 3.73209772e-03,
       9.30610790e-03])